In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark-3.0.3-bin-hadoop2.7'

In [2]:
from os.path import abspath

from pyspark.sql import SparkSession
from pyspark.sql import Row


yönetilen veritabanları ve tablolar için varsayılan konumum
spark-warehouse şuana kadar oluşturduğum hive tabloları burada depolanıyor
hive ile çalışırken sparkSesisonu hive desteği ile başlatmamız gerekir
Mevcut bir Hive dağıtımına sahip olmayan kullanıcılar yine de Hive desteğini etkinleştirebilir.ki benim şuanda kurduğum sparkta hive dağıtımına sahip değilim
warehouse taki veritabanının varsayılan konumunu belirtmek için spark.sql.warehouse.dir kullanıcam bunuda spark-datawarehouse
klasörüne eşitlicem

In [3]:
warehouse_location=abspath("spark-datawarehouse")#abspath fonksiyonu mevcut bulunduğum klasörü verir yani C:\Users\Mert4\
print(warehouse_location)
spark=SparkSession.builder.appName("spark sql hive integraton")\
                            .config("spark.sql.warehouse.dir",warehouse_location)\
                            .enableHiveSupport().getOrCreate()
#abspathi kullanmama gerek yok çünkü spark -datawarehouse klasörü ile bu notebook aynı klasörde

                

C:\Users\Mert4\spark-datawarehouse


In [4]:
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT,value STRING) USING hive")
#spark.sql("LOAD DATA LOCAL INPATH 'kv1.txt' INTO TABLE src")
#spark.sql("Truncate  TABLE src")
spark.sql("SELECT * FROM src").show()
spark.sql("SHOW TABLES").show()

+---+-------+
|key|  value|
+---+-------+
|238|val_238|
| 86| val_86|
|311|val_311|
| 27| val_27|
|165|val_165|
|409|val_409|
|255|val_255|
|278|val_278|
| 98| val_98|
|484|val_484|
|265|val_265|
|193|val_193|
|401|val_401|
|150|val_150|
|273|val_273|
|224|val_224|
|369|val_369|
| 66| val_66|
|128|val_128|
|213|val_213|
+---+-------+
only showing top 20 rows

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|    hive1|      false|
| default|    hive2|      false|
| default|    hive3|      false|
| default|   permat|      false|
| default|      src|      false|
| default|   tablo2|      false|
+--------+---------+-----------+



In [5]:
#bunu kendim yapmak istedim
#aynı txt dosyasını sc.textFile ile okuyup her kolonun değeri  ile ayrılıyordu buna göre map ile split ederek 
#her satırın kolon değerlerini dizi içerisine atıp sonrada map ile her elemana ulaşıp dizi içindeki değerlere 
# 2 tane değer var 1.sine key 2.sine value kolon ismini row ile verdik artık bunu toDF ile dfe dönüştürebiliriz
#dfe dönüştürdekten sonra geçici tablo yada hive tablosunuda dönüştürüp sql atabiliriz
sc=spark.sparkContext
rdd=sc.textFile("kv1.txt").map(lambda x :(x.split(""))).map(lambda x :Row(key=x[0],value=x[1]))
df=rdd.toDF()
df.show()
rdd.count()

+---+-------+
|key|  value|
+---+-------+
|238|val_238|
| 86| val_86|
|311|val_311|
| 27| val_27|
|165|val_165|
|409|val_409|
|255|val_255|
|278|val_278|
| 98| val_98|
|484|val_484|
|265|val_265|
|193|val_193|
|401|val_401|
|150|val_150|
|273|val_273|
|224|val_224|
|369|val_369|
| 66| val_66|
|128|val_128|
|213|val_213|
+---+-------+
only showing top 20 rows



500

In [7]:
spark.sql("SELECT count(*) FROM src").show() # aggregationları da kullanabiliriz

+--------+
|count(1)|
+--------+
|     500|
+--------+



In [8]:
sqlDF = spark.sql("SELECT key, value FROM src WHERE key < 10 ORDER BY key")
sqlDF.show()
#sql sorgusunun çıktısı dataframe olarak döndüğü için bu çıktıyı bir değişkene eşitleyip dataFrame API kullanabilirim




+---+-----+
|key|value|
+---+-----+
|  0|val_0|
|  0|val_0|
|  0|val_0|
|  2|val_2|
|  4|val_4|
|  5|val_5|
|  5|val_5|
|  5|val_5|
|  8|val_8|
|  9|val_9|
+---+-----+



In [9]:
#dfobjesi.rdd df i rdd ye dönüştürür
dfToRdd=sqlDF.rdd
dfToRdd.collect()
#rdd yi dfe çevirmek için rdd nin çıktısı aşağıdaki gibi olması lazım

[Row(key=0, value='val_0'),
 Row(key=0, value='val_0'),
 Row(key=0, value='val_0'),
 Row(key=2, value='val_2'),
 Row(key=4, value='val_4'),
 Row(key=5, value='val_5'),
 Row(key=5, value='val_5'),
 Row(key=5, value='val_5'),
 Row(key=8, value='val_8'),
 Row(key=9, value='val_9')]

sparksql yapılandırılmış veriyi işlemek için bir spark modülüdür.
Spark SQL, Apache Hive'da depolanan verilerin okunmasını ve yazılmasını da destekler


DataFrame'lerdeki öğeler, her bir sütuna sıraya göre erişmenizi sağlayan Row türündedir.

In [11]:
stringsDS = sqlDF.rdd.map(lambda row: "Key: %d, Value: %s" % (row.key, row.value))
for record in stringsDS.collect():
    print(record)

Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 2, Value: val_2
Key: 4, Value: val_4
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 8, Value: val_8
Key: 9, Value: val_9


In [12]:
Record = Row("key", "value")
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF.show()
recordsDF.createOrReplaceTempView("records")


+---+------+
|key| value|
+---+------+
|  1| val_1|
|  2| val_2|
|  3| val_3|
|  4| val_4|
|  5| val_5|
|  6| val_6|
|  7| val_7|
|  8| val_8|
|  9| val_9|
| 10|val_10|
| 11|val_11|
| 12|val_12|
| 13|val_13|
| 14|val_14|
| 15|val_15|
| 16|val_16|
| 17|val_17|
| 18|val_18|
| 19|val_19|
| 20|val_20|
+---+------+
only showing top 20 rows



In [51]:
spark.sql("SELECT * FROM records r JOIN src s ON r.key = s.key").show()

+---+------+---+------+
|key| value|key| value|
+---+------+---+------+
|  2| val_2|  2| val_2|
|  4| val_4|  4| val_4|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  8| val_8|  8| val_8|
|  9| val_9|  9| val_9|
| 10|val_10| 10|val_10|
| 11|val_11| 11|val_11|
| 12|val_12| 12|val_12|
| 12|val_12| 12|val_12|
| 15|val_15| 15|val_15|
| 15|val_15| 15|val_15|
| 17|val_17| 17|val_17|
| 18|val_18| 18|val_18|
| 18|val_18| 18|val_18|
| 19|val_19| 19|val_19|
| 20|val_20| 20|val_20|
| 24|val_24| 24|val_24|
| 24|val_24| 24|val_24|
+---+------+---+------+
only showing top 20 rows



In [14]:
rdd=sc.parallelize(range(1,101))
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100]

In [15]:
rdddf=rdd.map(lambda x :Row (key=x,value="val_"+str(x)))
denemedf=rdddf.toDF()
denemedf.show()

+---+------+
|key| value|
+---+------+
|  1| val_1|
|  2| val_2|
|  3| val_3|
|  4| val_4|
|  5| val_5|
|  6| val_6|
|  7| val_7|
|  8| val_8|
|  9| val_9|
| 10|val_10|
| 11|val_11|
| 12|val_12|
| 13|val_13|
| 14|val_14|
| 15|val_15|
| 16|val_16|
| 17|val_17|
| 18|val_18|
| 19|val_19|
| 20|val_20|
+---+------+
only showing top 20 rows



In [16]:
denemedf.createOrReplaceTempView("deneme")
spark.sql("SELECT * FROM deneme d JOIN src s ON d.key=s.key").show()

+---+------+---+------+
|key| value|key| value|
+---+------+---+------+
|  2| val_2|  2| val_2|
|  4| val_4|  4| val_4|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  8| val_8|  8| val_8|
|  9| val_9|  9| val_9|
| 10|val_10| 10|val_10|
| 11|val_11| 11|val_11|
| 12|val_12| 12|val_12|
| 12|val_12| 12|val_12|
| 15|val_15| 15|val_15|
| 15|val_15| 15|val_15|
| 17|val_17| 17|val_17|
| 18|val_18| 18|val_18|
| 18|val_18| 18|val_18|
| 19|val_19| 19|val_19|
| 20|val_20| 20|val_20|
| 24|val_24| 24|val_24|
| 24|val_24| 24|val_24|
+---+------+---+------+
only showing top 20 rows



In [66]:
stringsDS.collect()

['Key: 0, Value: val_0',
 'Key: 0, Value: val_0',
 'Key: 0, Value: val_0',
 'Key: 2, Value: val_2',
 'Key: 4, Value: val_4',
 'Key: 5, Value: val_5',
 'Key: 5, Value: val_5',
 'Key: 5, Value: val_5',
 'Key: 8, Value: val_8',
 'Key: 9, Value: val_9']